In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
# from pyfm import pylibfm

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
train = np.load('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/simple_npy/train2.npy')
test = np.load('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/simple_npy/test2.npy')
val = np.load('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/simple_npy/validation2.npy')

In [4]:
data = pd.read_csv('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/OneDrive_2018-02-25/train.csv')
data_test = pd.read_csv('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/OneDrive_2018-02-25/test.csv')
data_val = pd.read_csv('/Users/mengwei.zhang/Desktop/COMPGW02 WE/Couesework/data/OneDrive_2018-02-25/validation.csv')

In [4]:
data['click'].value_counts()/ data.shape[0]

0    0.999262
1    0.000738
Name: click, dtype: float64

In [5]:
data['click'].value_counts()

0    2429188
1       1793
Name: click, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430981 entries, 0 to 2430980
Data columns (total 25 columns):
click             int64
weekday           int64
hour              int64
bidid             object
userid            object
useragent         object
IP                object
region            int64
city              int64
adexchange        object
domain            object
url               object
urlid             object
slotid            object
slotwidth         int64
slotheight        int64
slotvisibility    object
slotformat        object
slotprice         int64
creative          object
bidprice          int64
payprice          int64
keypage           object
advertiser        int64
usertag           object
dtypes: int64(11), object(14)
memory usage: 463.7+ MB


## Constant Bidding

In [7]:
# # Caculate the calibrated CTR
# def Cali_CTR(p, w):

#     cali_ctr = p/(p+(1-p)/w)

#     return cali_ctr   

In [8]:
# # Caculate the root mean square rooot
# def rmse(predictions, targets):

#     differences = predictions - targets                       #the DIFFERENCEs.

#     differences_squared = differences ** 2                    #the SQUAREs of ^

#     mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^

#     rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^

#     return rmse_val     #get the ^

In [5]:
data['payprice'].describe()

count    2.430981e+06
mean     7.815142e+01
std      5.984636e+01
min      0.000000e+00
25%      3.300000e+01
50%      6.800000e+01
75%      9.900000e+01
max      3.000000e+02
Name: payprice, dtype: float64

In [13]:
data_val['payprice'].describe()

count    303925.000000
mean        273.053099
std          28.892861
min         227.000000
25%         241.000000
50%         294.000000
75%         300.000000
max         300.000000
Name: bidprice, dtype: float64

In [10]:
Payprice_val = val[:,12:13]
clicks_val = data_val['click']
clicks_val = clicks_val.as_matrix()


In [11]:
Payprice_train = train[:,12:13]
clicks_train = data['click']
clicks_train = clicks_train.as_matrix()


In [12]:
def const_evaluation (pBidprice, payprice, clicks, budget=6250):
    win_num = 0.
    total_cost = 0.
    click_num = 0.
    for i in range(0, len(payprice)):
        if pBidprice > payprice[i]:
            if total_cost + payprice[i] / 1000. > budget:
                break
            else:
                win_num = win_num + 1
                total_cost = total_cost + payprice[i] / 1000.
                click_num += clicks[i]
    ctr = click_num / win_num
    cpc = total_cost / click_num
    print('win_num {}, total_cost {}, click_num {}, ctr {}, cpc {}'.format(win_num, total_cost, click_num, ctr, cpc))
    return win_num, total_cost, click_num, ctr, cpc    

In [14]:
# Bid by average bid price of training data set
const_bidprice_train = 2.729620e+02
const_bidprice_val = 273.053099
win_num, total_cost, click_num, ctr, cpc = const_evaluation(const_bidprice_train, Payprice_train, clicks_train)
win_num, total_cost, click_num, ctr, cpc = const_evaluation(const_bidprice_val, Payprice_val, clicks_val)

win_num 81516.0, total_cost [6249.911000001028], click_num 56.0, ctr 0.0006869816968447912, cpc [111.60555357144692]
win_num 81589.0, total_cost [6249.865000000903], click_num 52.0, ctr 0.0006373408180024268, cpc [120.1897115384789]


In [15]:
# validation set, the bid price could be 78 or 80, which will give 68 clicks
new_const_bidprice = 50
for i in range (250):
    print("constant bidprice:", i+50)
    win_num, total_cost, click_num, ctr, cpc = const_evaluation(new_const_bidprice, Payprice_val, clicks_val)
    print(new_const_bidprice)
    new_const_bidprice += 1

constant bidprice: 50
win_num 101750.0, total_cost [2459.3669999997373], click_num 37.0, ctr 0.0003636363636363636, cpc [66.46937837837127]
50
constant bidprice: 51
win_num 115493.0, total_cost [3146.517000000476], click_num 43.0, ctr 0.00037231693695721817, cpc [73.17481395349944]
51
constant bidprice: 52
win_num 117911.0, total_cost [3269.835000000472], click_num 46.0, ctr 0.00039012475511190643, cpc [71.08336956522766]
52
constant bidprice: 53
win_num 123022.0, total_cost [3535.607000000829], click_num 48.0, ctr 0.0003901741151989075, cpc [73.65847916668393]
53
constant bidprice: 54
win_num 124073.0, total_cost [3591.3100000008567], click_num 48.0, ctr 0.00038686902065719375, cpc [74.81895833335118]
54
constant bidprice: 55
win_num 125005.0, total_cost [3641.6380000009367], click_num 48.0, ctr 0.00038398464061437543, cpc [75.86745833335284]
55
constant bidprice: 56
win_num 132801.0, total_cost [4070.418000000634], click_num 49.0, ctr 0.0003689731252023705, cpc [83.06975510205376]
56

win_num 122433.0, total_cost [6249.971000001856], click_num 58.0, ctr 0.00047372848823438123, cpc [107.75812068968717]
108
constant bidprice: 109
win_num 121999.0, total_cost [6249.999000001878], click_num 58.0, ctr 0.00047541373289945, cpc [107.75860344830824]
109
constant bidprice: 110
win_num 121680.0, total_cost [6249.942000001907], click_num 57.0, ctr 0.0004684418145956607, cpc [109.64810526319135]
110
constant bidprice: 111
win_num 121348.0, total_cost [6249.941000001872], click_num 57.0, ctr 0.0004697234400237334, cpc [109.6480877193311]
111
constant bidprice: 112
win_num 120988.0, total_cost [6249.991000001835], click_num 56.0, ctr 0.0004628558204119417, cpc [111.60698214288992]
112
constant bidprice: 113
win_num 120516.0, total_cost [6249.950000001837], click_num 57.0, ctr 0.00047296624514587274, cpc [109.64824561406732]
113
constant bidprice: 114
win_num 120114.0, total_cost [6249.95300000184], click_num 57.0, ctr 0.0004745491782806334, cpc [109.64829824564632]
114
constant b

win_num 100939.0, total_cost [6249.973000001289], click_num 57.0, ctr 0.0005646974905636077, cpc [109.64864912282962]
165
constant bidprice: 166
win_num 100697.0, total_cost [6249.851000001285], click_num 57.0, ctr 0.00056605459944189, cpc [109.64650877195237]
166
constant bidprice: 167
win_num 99839.0, total_cost [6249.993000001354], click_num 57.0, ctr 0.0005709191798796061, cpc [109.64900000002375]
167
constant bidprice: 168
win_num 99507.0, total_cost [6249.972000001378], click_num 57.0, ctr 0.0005728240224305828, cpc [109.64863157897153]
168
constant bidprice: 169
win_num 99143.0, total_cost [6249.897000001354], click_num 57.0, ctr 0.0005749271254652371, cpc [109.64731578949744]
169
constant bidprice: 170
win_num 98872.0, total_cost [6249.948000001349], click_num 58.0, ctr 0.0005866170402136096, cpc [107.7577241379543]
170
constant bidprice: 171
win_num 98726.0, total_cost [6249.974000001349], click_num 58.0, ctr 0.0005874845532078683, cpc [107.75817241381637]
171
constant bidpric

win_num 86807.0, total_cost [6249.980000001025], click_num 55.0, ctr 0.0006335894570714344, cpc [113.63600000001863]
223
constant bidprice: 224
win_num 86665.0, total_cost [6249.993000001006], click_num 55.0, ctr 0.000634627588992096, cpc [113.63623636365466]
224
constant bidprice: 225
win_num 86595.0, total_cost [6249.934000001007], click_num 55.0, ctr 0.0006351405970321612, cpc [113.63516363638195]
225
constant bidprice: 226
win_num 86058.0, total_cost [6249.935000001021], click_num 54.0, ctr 0.0006274837900020916, cpc [115.73953703705594]
226
constant bidprice: 227
win_num 85742.0, total_cost [6249.972000000997], click_num 54.0, ctr 0.0006297963658417112, cpc [115.74022222224067]
227
constant bidprice: 228
win_num 85596.0, total_cost [6249.991000000984], click_num 53.0, ctr 0.0006191878125146034, cpc [117.92435849058461]
228
constant bidprice: 229
win_num 85455.0, total_cost [6249.988000000983], click_num 53.0, ctr 0.0006202094669709204, cpc [117.92430188681101]
229
constant bidpric

win_num 81094.0, total_cost [6249.988000000899], click_num 52.0, ctr 0.0006412311638345624, cpc [120.1920769230942]
281
constant bidprice: 282
win_num 80638.0, total_cost [6249.985000000874], click_num 51.0, ctr 0.0006324561621071952, cpc [122.54872549021322]
282
constant bidprice: 283
win_num 80591.0, total_cost [6249.984000000881], click_num 51.0, ctr 0.0006328250052735417, cpc [122.54870588237021]
283
constant bidprice: 284
win_num 80530.0, total_cost [6249.967000000881], click_num 51.0, ctr 0.0006333043586241152, cpc [122.54837254903688]
284
constant bidprice: 285
win_num 80484.0, total_cost [6249.97200000088], click_num 51.0, ctr 0.0006336663187714328, cpc [122.54847058825256]
285
constant bidprice: 286
win_num 80437.0, total_cost [6249.891000000871], click_num 51.0, ctr 0.0006340365752079267, cpc [122.54688235295825]
286
constant bidprice: 287
win_num 80357.0, total_cost [6249.968000000877], click_num 51.0, ctr 0.0006346677949649689, cpc [122.54839215687993]
287
constant bidprice

In [16]:
# train set, the bid price could be 51 or 52, which will give 86 clicks; 50 will give 102
new_const_bidprice = 50
for i in range (250):
    print("constant bidprice:", i+50)
    win_num, total_cost, click_num, ctr, cpc = const_evaluation(new_const_bidprice, Payprice_train, clicks_train)
    print(new_const_bidprice)
    new_const_bidprice += 1

constant bidprice: 50
win_num 257395.0, total_cost [6249.990000004239], click_num 102.0, ctr 0.00039627809398006957, cpc [61.27441176474744]
50
constant bidprice: 51
win_num 228609.0, total_cost [6249.988000006472], click_num 86.0, ctr 0.0003761881640705309, cpc [72.6742790698427]
51
constant bidprice: 52
win_num 224527.0, total_cost [6249.993000006708], click_num 86.0, ctr 0.00038302743099938985, cpc [72.67433720938033]
52
constant bidprice: 53
win_num 216728.0, total_cost [6249.982000005474], click_num 79.0, ctr 0.0003645121996234912, cpc [79.11369620260093]
53
constant bidprice: 54
win_num 215141.0, total_cost [6249.991000005292], click_num 79.0, ctr 0.00036720104489613786, cpc [79.11381012664927]
54
constant bidprice: 55
win_num 213859.0, total_cost [6249.971000005345], click_num 78.0, ctr 0.00036472629162205, cpc [80.12783333340185]
55
constant bidprice: 56
win_num 203324.0, total_cost [6249.97500000547], click_num 76.0, ctr 0.0003737876492691468, cpc [82.23651315796671]
56
consta

win_num 121978.0, total_cost [6249.979000001814], click_num 73.0, ctr 0.0005984685763006444, cpc [85.61615068495635]
108
constant bidprice: 109
win_num 121499.0, total_cost [6249.980000001856], click_num 75.0, ctr 0.0006172890311854418, cpc [83.33306666669141]
109
constant bidprice: 110
win_num 121174.0, total_cost [6249.939000001891], click_num 75.0, ctr 0.0006189446580949708, cpc [83.33252000002521]
110
constant bidprice: 111
win_num 120891.0, total_cost [6249.948000001866], click_num 75.0, ctr 0.0006203935776856838, cpc [83.33264000002488]
111
constant bidprice: 112
win_num 120523.0, total_cost [6249.912000001835], click_num 74.0, ctr 0.0006139906905735835, cpc [84.45827027029506]
112
constant bidprice: 113
win_num 120090.0, total_cost [6249.941000001847], click_num 75.0, ctr 0.0006245316012990258, cpc [83.3325466666913]
113
constant bidprice: 114
win_num 119695.0, total_cost [6249.977000001848], click_num 75.0, ctr 0.0006265925894983082, cpc [83.33302666669131]
114
constant bidpric

win_num 100557.0, total_cost [6249.981000001255], click_num 72.0, ctr 0.0007160118141949343, cpc [86.8052916666841]
165
constant bidprice: 166
win_num 100368.0, total_cost [6249.986000001248], click_num 72.0, ctr 0.0007173601147776184, cpc [86.80536111112843]
166
constant bidprice: 167
win_num 99742.0, total_cost [6249.948000001301], click_num 70.0, ctr 0.000701810671532554, cpc [89.28497142859001]
167
constant bidprice: 168
win_num 99407.0, total_cost [6249.918000001298], click_num 69.0, ctr 0.0006941161085235446, cpc [90.57852173914925]
168
constant bidprice: 169
win_num 99067.0, total_cost [6249.977000001266], click_num 66.0, ctr 0.0006662157933519739, cpc [94.69662121214039]
169
constant bidprice: 170
win_num 98813.0, total_cost [6249.9640000012405], click_num 66.0, ctr 0.0006679283090281643, cpc [94.69642424244304]
170
constant bidprice: 171
win_num 98671.0, total_cost [6249.990000001242], click_num 66.0, ctr 0.0006688895420133576, cpc [94.696818181837]
171
constant bidprice: 172


win_num 86575.0, total_cost [6249.987000001086], click_num 57.0, ctr 0.0006583886803349697, cpc [109.64889473686115]
223
constant bidprice: 224
win_num 86473.0, total_cost [6249.869000001082], click_num 57.0, ctr 0.000659165288587189, cpc [109.64682456142249]
224
constant bidprice: 225
win_num 86434.0, total_cost [6249.9430000010825], click_num 57.0, ctr 0.0006594627114330009, cpc [109.64812280703653]
225
constant bidprice: 226
win_num 85883.0, total_cost [6249.996000001104], click_num 57.0, ctr 0.0006636936297055296, cpc [109.64905263159832]
226
constant bidprice: 227
win_num 85581.0, total_cost [6249.979000001074], click_num 57.0, ctr 0.0006660356854909384, cpc [109.64875438598376]
227
constant bidprice: 228
win_num 85443.0, total_cost [6249.8480000010595], click_num 57.0, ctr 0.00066711140760507, cpc [109.64645614036947]
228
constant bidprice: 229
win_num 85344.0, total_cost [6249.980000001059], click_num 58.0, ctr 0.0006796025496812898, cpc [107.75827586208723]
229
constant bidpric

win_num 81180.0, total_cost [6249.976000001025], click_num 56.0, ctr 0.0006898250800689825, cpc [111.60671428573258]
280
constant bidprice: 281
win_num 81062.0, total_cost [6249.997000001018], click_num 56.0, ctr 0.0006908292418149071, cpc [111.60708928573247]
281
constant bidprice: 282
win_num 80656.0, total_cost [6249.967000000991], click_num 56.0, ctr 0.0006943066851815116, cpc [111.60655357144627]
282
constant bidprice: 283
win_num 80602.0, total_cost [6249.868000000992], click_num 56.0, ctr 0.0006947718418897794, cpc [111.60478571430342]
283
constant bidprice: 284
win_num 80522.0, total_cost [6249.932000000988], click_num 56.0, ctr 0.0006954621097339858, cpc [111.60592857144623]
284
constant bidprice: 285
win_num 80475.0, total_cost [6249.843000000985], click_num 56.0, ctr 0.0006958682820751786, cpc [111.60433928573187]
285
constant bidprice: 286
win_num 80432.0, total_cost [6249.863000000983], click_num 55.0, ctr 0.0006838074398249453, cpc [113.63387272729061]
286
constant bidpri

## Random Bidding 

In [17]:
def evaluation (pBidprice, payprice, clicks, budget=6250):
    win_num = 0.
    total_cost = 0.
    click_num = 0.
    for i in range(0, len(payprice)):
        if pBidprice[i] >= payprice[i]:
            if total_cost + payprice[i] / 1000. > budget:
                break
            win_num = win_num + 1
            total_cost = total_cost + payprice[i] / 1000.
            click_num += clicks[i]
    ctr = click_num / win_num
    cpc = total_cost / click_num
    print('win_num {}, total_cost {}, click_num {}, ctr {}, cpc {}'.format(win_num, total_cost, click_num, ctr, cpc), '\n')
    return win_num, total_cost, click_num, ctr, cpc
        

In [18]:
# Random bidding betwen
import random
random.seed(77)
random_bidprice =[]

T_win_num = 0.
T_total_cost = 0.
T_click_num = 0.
T_ctr = 0.
T_cpc =0.

for i in range(100):
    random_bidprice = np.random.randint(200, high=300, size=len(Payprice_val), dtype='l')
    print('Bid value is:', random_bidprice[i]) 
    win_num, total_cost, click_num, ctr, cpc = evaluation(random_bidprice, Payprice_val, clicks_val)
    T_win_num = T_win_num + win_num
    T_total_cost = T_total_cost + total_cost
    T_click_num = T_click_num + click_num
    T_ctr = T_ctr + ctr
    T_cpc = T_cpc + cpc
    

print('The average of 100 random result:\n win: %f, total_cost: %f, click_num: %d, ctr: %f, cpc: %f'
      %(T_win_num/100, T_total_cost/100,T_click_num/100,T_ctr/100,T_cpc/100))

Bid value is: 263
win_num 83915.0, total_cost [6249.936000000952], click_num 53.0, ctr 0.0006315914913900972, cpc [117.92332075473495] 

Bid value is: 227
win_num 83863.0, total_cost [6249.997000000958], click_num 54.0, ctr 0.0006439073250420329, cpc [115.74068518520292] 

Bid value is: 292
win_num 83790.0, total_cost [6249.960000000952], click_num 53.0, ctr 0.0006325337152404822, cpc [117.92377358492362] 

Bid value is: 246
win_num 83946.0, total_cost [6249.990000000951], click_num 53.0, ctr 0.0006313582541157411, cpc [117.92433962265946] 

Bid value is: 233
win_num 84022.0, total_cost [6249.960000000953], click_num 54.0, ctr 0.000642688819594868, cpc [115.74000000001764] 

Bid value is: 290
win_num 83898.0, total_cost [6249.943000000958], click_num 53.0, ctr 0.0006317194688788767, cpc [117.92345283020676] 

Bid value is: 234
win_num 83871.0, total_cost [6249.909000000948], click_num 53.0, ctr 0.0006319228338758331, cpc [117.92281132077261] 

Bid value is: 298
win_num 83873.0, total_c

win_num 83855.0, total_cost [6249.956000000943], click_num 54.0, ctr 0.0006439687555900065, cpc [115.7399259259434] 

Bid value is: 241
win_num 83848.0, total_cost [6249.985000000949], click_num 54.0, ctr 0.000644022516935407, cpc [115.74046296298054] 

Bid value is: 258
win_num 84009.0, total_cost [6249.914000000958], click_num 53.0, ctr 0.0006308847861538645, cpc [117.92290566039544] 

Bid value is: 283
win_num 84029.0, total_cost [6249.982000000951], click_num 54.0, ctr 0.0006426352806769091, cpc [115.74040740742502] 

Bid value is: 291
win_num 83961.0, total_cost [6249.928000000949], click_num 54.0, ctr 0.0006431557508843392, cpc [115.73940740742499] 

Bid value is: 261
win_num 83875.0, total_cost [6249.987000000953], click_num 53.0, ctr 0.0006318926974664679, cpc [117.92428301888592] 

Bid value is: 263
win_num 83955.0, total_cost [6249.966000000964], click_num 53.0, ctr 0.0006312905723304151, cpc [117.92388679247102] 

Bid value is: 223
win_num 83910.0, total_cost [6249.967000000